<a id="intro"></a>
<br>
<br>

# Create & place receivers on the surface of building facades


This notebook is dedicated to describing how to produce a point-layer of receivers placed on the surface of building facades. The placement of the receivers follows the pattern described by the [German regulation VBEB (Vorläufige Berechnungsmethode zur Ermittlung der Belastetenzahlen durch Umgebungslärm (VBEB), Federal Ministry of the
Environment ( 07.02.2007))](https://www.umweltbundesamt.de/sites/default/files/medien/pdfs/VBEB.pdf) and recommended by [Kefalopoulos et al. (2012)](https://publications.jrc.ec.europa.eu/repository/handle/JRC72550) in the Common Noise Assessment Methods in Europe (CNOSSOS-EU) list of recommendations.

The recommendations for implementing the CNOSSOS noise propagation method require that receivers are placed at 4m above the ground height and at a 2m distance from the building facade. The horizontal resolution of the receivers is explained below with the help of Fig.1 (borrowed from [Kefalopoulos et al. (2012)](https://publications.jrc.ec.europa.eu/repository/handle/JRC72550)).

<center><img src="https://github.com/KarolinaPntzt/receivers/blob/main/img/horizontal_placement_of_receivers_method.JPG?raw=True" width=50%)></img></center>
<br>
<br>

1. Segments of a length of more than 5 m are split up into regular intervals of the longest possible length, but less than or equal to 5 m. Receiver points are placed in the middle of each regular interval (blue/green).
<br><br>
2. Remaining segments above a length of 2.5 m are represented by one receiver point in the middle of each segment (pink).
<br><br>
3. Remaining subsequent segments with a total length of more than 5 m are treated as polyline objects in a manner similar to that described in a) and b) (red).

<br>
<br>

<a id="import_mods"></a>
<br>
<br>

## 1. Import modules
<br>
<br>

In [1]:
#Import modules:
import arcpy
import math
import pandas as pd

<a id="paths_input"></a>
<br>
<br>

## 2. Set paths to input data 

<br>
<br>

In [2]:
#Set worskpace:
arcpy.env.workspace = "C:\\Users\\karol\\Downloads\\b_fps_all\\test\\"
#C:\Users\karol\Downloads\b_fps_all\b_fps_all_simplified.shp
#Set paths to input/output data:
b_fp_path = "bf_single_poly_simplified.shp"
r_out_path = "C:\\Users\\karol\\Desktop\\noise_sim\\receivers\\" #buffer fixed

In [3]:
#Convert 3CIM building ground surface building footprints to 2D building footprints using the Multipatch Footprint tool
#shapefile with building footprints
#convert poly to line using https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/polygon-to-line.htm
#split line to points: https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/feature-vertices-to-points.htm

<a id="help_funcs"></a>
<br>
<br>

## 3. Help functions

<br>
<br>

In [4]:
#Create class for storing vertex info:
class Vertex:
    
    #Constructor
    def __init__(self, id, b_id, coord_x, coord_y, coord_z):
        self.id = str(id)
        self.building_id = b_id
        self.coord_x = coord_x
        self.coord_y = coord_y
        self.coord_z = coord_z

In [5]:
#Create class for storing Line segment info:
class Line_segment:
    
    #Import module:
    import math
    
    
    #Constructor
    def __init__(self, id, building_id, vertex1, vertex2):
        self.id = str(id)
        #self.facade_id = facade_id
        self.building_id = building_id
        self.vertex1 = Vertex(vertex1[0], vertex1[1], vertex1[2], vertex1[3], vertex1[4])
        self.vertex2 = Vertex(vertex2[0], vertex2[1], vertex2[2], vertex2[3], vertex1[4])
        self.length = self.get_length()

        
   #Function for computing the length of a line segment:    
    def get_length(self):
        
        #Compute the length of the line segment by applying the Euclidean distance equation:
        line_length = math.sqrt(pow(self.vertex2.coord_x - self.vertex1.coord_x, 2) + pow(self.vertex2.coord_y - self.vertex1.coord_y, 2))
        
        #Return value for line segment length:
        return line_length     
    

In [6]:
#Function that computes the receiver coordinates (x,y,z)
#as the middle point of a given line segment.
#The following parameters are used as input:
# 1. line segment (line object) 
# 2. s_midpoint (distance between vertex1 and middle point of line segment - in meters)
def get_receiver_coords(line_segment, s_midpoint):
    
    #Check if the x-coords of the line-segment vertices are the same:
    if (line_segment.vertex1.coord_x == line_segment.vertex2.coord_x):
        
        #Since our receiver point is part of the same line:
        x_receiver = line_segment.vertex1.coord_x
        
        #Try to find the receiver y-coord from the distance equations:
        #between line_segment.vertex1 & receiver as well as line_segment.vertex2 & receiver
        
        #The 1st distance eqution will provide 2 solutions for the receiver's y-coord:
        y_receiver_dist_v1_a = line_segment.vertex1.coord_y + s_midpoint
        y_receiver_dist_v1_b = line_segment.vertex1.coord_y - s_midpoint
    
        #The 2nd distance eqution will also provide 2 solutions for the receiver's y-coord:
        y_receiver_dist_v2_a = line_segment.vertex2.coord_y + (line_segment.length - s_midpoint)
        y_receiver_dist_v2_b = line_segment.vertex2.coord_y - (line_segment.length - s_midpoint)
        
        #Now, we need to find which y-coord solutions are identical from the previous 2 equations:
        if (y_receiver_dist_v1_a == y_receiver_dist_v2_a):
            y_receiver = y_receiver_dist_v1_a
            
        elif (y_receiver_dist_v1_a == y_receiver_dist_v2_b):
            y_receiver = y_receiver_dist_v1_a
            
        elif (y_receiver_dist_v1_b == y_receiver_dist_v2_a):
            y_receiver = y_receiver_dist_v1_b
            
        elif (y_receiver_dist_v1_b == y_receiver_dist_v2_b):
            y_receiver = y_receiver_dist_v1_b
            
        else:
            y_receiver = 0
    
    
    #If the line-segment does not contain vertices with identical x-coordinates:
    elif (line_segment.vertex1.coord_x != line_segment.vertex2.coord_x):
        
        #Compute the slope:
        slope = (line_segment.vertex2.coord_y - line_segment.vertex1.coord_y)/(line_segment.vertex2.coord_x - line_segment.vertex1.coord_x)
        
        #Compute length of remaining segment after subtracting length between vertex1 and receiver point (s_midpoint)
        remaining_length_line_segm = line_segment.length - s_midpoint
        
        #Get the equation from line-segment vertices:
        #y = slope * (x - line_segment.vertex1.coord_x) + line_segment.vertex1.coord_y
        
        #Since the receiver-point is part of the line-segment, its coordinates will fit the aforementioned line equation:
        #y_receiver = slope * (x_receiver - line_segment.vertex1.coord_x) + line_segment.vertex1.coord_y
        
        
        #Moreover, we know the length of the line-subsegment between vertex1 and the receiver-point.
        #From solving this equation for the receiver's x-coord, we get 2 possible solutions:
        receiver_v1_dist_coord_xa = math.sqrt(pow(s_midpoint,2)/(1 + pow(slope,2))) + line_segment.vertex1.coord_x
        receiver_v1_dist_coord_xb = line_segment.vertex1.coord_x - math.sqrt(pow(s_midpoint,2)/(1 + pow(slope,2)))
        
        #Additionally, we know the length of the remaining line subsegment
        #after subtracting the s-midpoint from the segment's length.
        #From solving this equation, we will get 2 possible solutions for the receiver's x-coord:
        receiver_v2_dist_coord_xa = line_segment.vertex2.coord_x - math.sqrt(pow(remaining_length_line_segm,2)/(1 + pow(slope,2)))
        receiver_v2_dist_coord_xb = line_segment.vertex2.coord_x + math.sqrt(pow(remaining_length_line_segm,2)/(1 + pow(slope,2)))
        
        #Now, we need too find which solution provvides the same value in both equations:
        if (receiver_v1_dist_coord_xa == receiver_v2_dist_coord_xa):
            x_receiver = receiver_v1_dist_coord_xa
            
        elif(receiver_v1_dist_coord_xa == receiver_v2_dist_coord_xb):
            x_receiver = receiver_v1_dist_coord_xa
            
        elif(receiver_v1_dist_coord_xb == receiver_v2_dist_coord_xa):
            x_receiver = receiver_v1_dist_coord_xb
            
        elif(receiver_v1_dist_coord_xb == receiver_v2_dist_coord_xb):
            x_receiver = receiver_v1_dist_coord_xb
            
        else:
            x_receiver = 0
        
        #Now that we have found the receiver's x-coord, 
        #we can replace this value in the original line equation
        #to obtain the receiver's y-coord:
        y_receiver = slope * (x_receiver - line_segment.vertex1.coord_x) + line_segment.vertex1.coord_y
        
    
    #If none of the above are true, return the following:
    else:
        x_receiver = 0
        y_receiver = 0
        z_receiver = 0
       
    
    #Finally, we can create a new point object for the receiver and 
    #populate its attributes with the computed values:
    receiver = Vertex(line_segment.id, line_segment.building_id, x_receiver, y_receiver, line_segment.vertex1.coord_z)
    
    #Return vertex with receiver info:
    return receiver
            
            
    

In [7]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) and returns
#the column names of the shapefile:
def get_field_names(fc):
    
    #Get a list of fieldnames for input shapefile:
    f_names = [f.name for f in arcpy.ListFields(fc)]
    
    #Return list with fieldnames:
    return f_names

In [8]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) and returns
#the content of the corresponding Attribute Table:
def get_TOC(fc):
    
    #Get a list of field-names:
    f_names = [f.name for f in arcpy.ListFields(fc)]
    
    #Open shapefile and print every row in TOC:
    with arcpy.da.SearchCursor(fc, f_names) as cursor:
        
        for row in cursor:
            
            print(row)

In [9]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) together with 
#the column name whose values we want to extract and a boolean 
#(unique=True returns only unique values in column) and returns
#the content of the corresponding column in a list:
def get_TOC_column(fc, colname, unique):
    
    #Create a list to store the column values:
    ls = []
    
    #Open shapefile:
    with arcpy.da.SearchCursor(fc, colname) as cursor:
        
        #Loop through every row in shapefile:
        for row in cursor:
            
            #If user wants the output list to only include unique values:
            if (unique):
                
                #Check that values is not already stored in the list:
                if row[0] not in ls:
                    
                    #Append value:
                    ls.append(row[0])
                    
            else:
                
                #Append column value to list:
                ls.append(row[0])
    
    #return list:
    return ls

In [10]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) together with 
#the column names whose values we want to extract and a boolean 
#(unique=True returns only unique values in row) and returns
#the content of the corresponding columns in a list:
def get_TOC_columns(fc, colnames, unique):
    
    #Create a list to store the column values:
    ls = []
    
    #Open shapefile:
    with arcpy.da.SearchCursor(fc, colnames) as cursor:
        
        #Loop through every row in shapefile:
        for row in cursor:
            
            #If user wants the output list to only include 
            #unique combination of row values:
            if (unique):
                
                #Check that row values are not already stored in the list:
                if row not in ls:
                    
                    #Append row values:
                    ls.append(row)
                    
            else:
                
                #Append row value to list:
                ls.append(row)
    
    #return list:
    return ls

<a id="main_func"></a>
<br>
<br>

## 4. Main function

<br>
<br>

In [11]:
#Set receiver height (measured as height above ground level):
rec_height = 1.5

#Set buffer size (expressed as distance from facade in meters):
buffer_size = 0.01

#Create a list to store the receiver info:
receiver_ls = []
##########################################################################################################################

#Create a buffer around every building footprint:
arcpy.analysis.Buffer(b_fp_path, r_out_path+"fp_buffer.shp", str(buffer_size)+" Meters", "FULL")

#Simplify the buffer polygon:
arcpy.cartography.SimplifyPolygon(r_out_path+"fp_buffer.shp", r_out_path+"fp_buffer_simplified.shp", "POINT_REMOVE", 0.2)

#Split building fp polygons at vertices to get lines:
arcpy.management.SplitLine(r_out_path+"fp_buffer_simplified.shp", r_out_path+"fp_lines.shp")

#Get start- and end-point coords for every building fp line:
arcpy.management.FeatureVerticesToPoints(r_out_path+"fp_lines.shp", r_out_path+"fp_points.shp", "BOTH_ENDS")

get_field_names(r_out_path+"fp_points.shp") 

#Compute the length of every line segment 

['FID',
 'Shape',
 'gml_id',
 'gml_parent',
 'height_mag',
 'name_id',
 'min_height',
 'max_height',
 'area',
 'InPoly_FID',
 'SimPgnFlag',
 'MaxSimpTol',
 'MinSimpTol',
 'BUFF_DIST',
 'ORIG_FID']

In [12]:
#Store building footprint vertices in pandas dataframe:
b_fp_pts_df = pd.DataFrame(data = get_TOC_columns(r_out_path+"fp_points.shp", ['FID', 'Shape', 'ORIG_FID'], False),
                           columns = ['FID', 'Shape', 'ORIG_FID'])

b_fp_pts_df

,FID,Shape,ORIG_FID
0,0,"(117330.55172742065, 6162040.951653756)",0
1,1,"(117325.12926662723, 6162031.207860586)",0
2,2,"(117325.12926662723, 6162031.207860586)",1
3,3,"(117322.88526928425, 6162032.449797846)",1
4,4,"(117322.88526928425, 6162032.449797846)",2
5,5,"(117316.18480283674, 6162020.42266926)",2
6,6,"(117316.18480283674, 6162020.42266926)",3
7,7,"(117318.42116860766, 6162019.166360417)",3
8,8,"(117318.42116860766, 6162019.166360417)",4
9,9,"(117312.99870027229, 6162009.411870521)",4


In [13]:
###############################################################
#Get building_ID & building min_height from the building fp row:
###############################################################
b_id = get_TOC_column(b_fp_path, "gml_parent", True)[0]
b_min_height = get_TOC_column(b_fp_path, "min_height", True)[0]
###############################################################

#Create a list to store line-objects for every line segment per building footprint:
line_obj_ls = [Line_segment(b_fp_pts_df.ORIG_FID.iloc[i], 
                            b_id,
                            (1, b_id, b_fp_pts_df.Shape.iloc[i][0], b_fp_pts_df.Shape.iloc[i][1], b_min_height + rec_height),
                            (2, b_id, b_fp_pts_df.Shape.iloc[i+1][0], b_fp_pts_df.Shape.iloc[i+1][1], b_min_height + rec_height))
               for i in range(len(b_fp_pts_df)-1) 
               if b_fp_pts_df.ORIG_FID.iloc[i+1]==b_fp_pts_df.ORIG_FID.iloc[i]]

len(line_obj_ls)

20

In [14]:
#Function that takes a line object as input, cuts it in the middle
#and returns the resulting 2 line objects:
def cut_line(line_obj):
            
    #Get x-, y-, z-coords for the middle point of the input line-segment:
    middle_point = get_receiver_coords(line_obj, round(line_obj.length/2,1))

    #Create two new line segments:
    line_sub_segment_A = Line_segment(line_obj.id+"A", line_obj.building_id, 
                                      (line_obj.vertex1.id, line_obj.vertex1.building_id,
                                       line_obj.vertex1.coord_x, line_obj.vertex1.coord_y, line_obj.vertex1.coord_z), 
                                      (middle_point.id, middle_point.building_id,
                                       middle_point.coord_x, middle_point.coord_y, middle_point.coord_z))
    
    line_sub_segment_B = Line_segment(line_obj.id+"B", line_obj.building_id,
                                      (middle_point.id, middle_point.building_id,
                                       middle_point.coord_x, middle_point.coord_y, middle_point.coord_z), 
                                      (line_obj.vertex2.id, line_obj.vertex2.building_id,
                                       line_obj.vertex2.coord_x, line_obj.vertex2.coord_y, line_obj.vertex2.coord_z))
    
    #Return line sub-segments:
    return line_sub_segment_A, line_sub_segment_B

In [15]:
#Create a list to store consequitive line segments:
l_conseq_ls = []

for item in line_obj_ls:
    print("\n\n\n\n")
    print(item.id)
    print(item.length)
    
    # Check length of line-segment #
    if ((item.length>2.5) & (item.length<=5.0)):
        
        #print("Final line length: ", line_subsegment_ls[-1].length)
        print("A receiver is placed on a short line..")
        
        #Compute the receiver coords for the given line segment and add it to the list:
        receiver_ls.append(get_receiver_coords(item, round(item.length/2,1)))
        
        
    #If lenght of line segment is > 5m:    
    elif (item.length > 5.0):
        
        #Create a list to store line sub-segments:
        line_subsegment_ls = []
                
        #Create and initialize variable that stores the line-segment length.
        line_segment_length = item.length
        
        #Create and initialize counter:
        counter = 0
        
        
        #Repeat the process
        while(line_segment_length > 5.0):
            
            #Break line-segment to 2 line-subsegments and calculate length
            line_segment_length = line_segment_length/2
            
            print("Lenght of line segment before split: ", line_segment_length)
            
            #If this is the 1st time the loop runs:
            if (counter==0):
                
                print("while-loop executed for the 1st time")
                print("Line length: ", item.length)
                
                #Create two new line-segments:
                line_sub_segment_A, line_sub_segment_B = cut_line(item)
                
                print("Line length of 1st subsegment: ", line_sub_segment_A.length)
                
                #Check the length of the line sub-segments (since they have the same length, we check only one):
                if (line_sub_segment_A.length <= 5):
                    
                    #print("Final line length: ", line_subsegment_ls[-1].length)
                    print("A receiver is placed in a long line..")

                    #Compute the receiver coords for both line sub-segments:
                    receiver_ls.append(get_receiver_coords(line_sub_segment_A, round(line_sub_segment_A.length/2,1)))
                    receiver_ls.append(get_receiver_coords(line_sub_segment_B, round(line_sub_segment_B.length/2,1)))

                #If the length of the line sub-segment is still > 5m:
                else:
                    
                    #Append line subsegments to list:
                    line_subsegment_ls.append(line_sub_segment_A)
                    line_subsegment_ls.append(line_sub_segment_B)
                    
                    #Increase counter:
                    counter = counter + 1
                    
                    
                    
            #If this is not the 1st time the loop runs:
            else:
                
                print("\ncounter: ", counter)
                
                #Copy current list of line subsegments to new list:
                current_line_subsegment_ls = line_subsegment_ls
                
                #Reset list of line subsegments to new list:
                line_subsegment_ls = []
                
                #Create two new line subsegments for every line subsegment in the list:
                for line in current_line_subsegment_ls:
                    
                    print("Line ID: ", line.id)
                    print("Line length:", line.length)
                    
                    line_sub_segment_A, line_sub_segment_B = cut_line(line)
                    
                    #Append new line subsegments to list:
                    line_subsegment_ls.append(line_sub_segment_A)
                    line_subsegment_ls.append(line_sub_segment_B)
                    

                #Check the length of the line sub-segments (since they have the same length, we check only one):
                if (line_subsegment_ls[-1].length <= 5):
                    
                    print("Final line length: ", line_subsegment_ls[-1].length)
                    print("A receiver is placed in a long line..")
                    
                    #Compute the receiver coords for all line sub-segments in the list:
                    for line_ss in line_subsegment_ls:
                        receiver_ls.append(get_receiver_coords(line_ss, round(line_ss.length/2,1)))
      

                #If the length of the line sub-segment is still > 5m:
                else:
                    counter = counter + 1
                    
                
     
    #If current the current line segment's length is less than 2.5m:
    elif (item.length<=2.5):
        
        #print("Final line length: ", line_subsegment_ls[-1].length)
        print("A receiver is placed in a long line..")
        
        #Compute the receiver coords for the given line segment and add it to the list:
        receiver_ls.append(get_receiver_coords(item, item.length/2))
        
    else:
        print('Error!\nLine length could not be processed...')
        
    print()






0
11.15099037732711
Lenght of line segment before split:  5.575495188663555
while-loop executed for the 1st time
Line length:  11.15099037732711
Line length of 1st subsegment:  5.599999999720331
Lenght of line segment before split:  2.7877475943317775

counter:  1
Line ID:  0A
Line length: 5.599999999720331
Line ID:  0B
Line length: 5.550990377606779
Final line length:  2.750990377778457
A receiver is placed in a long line..






1
2.56474798634538
A receiver is placed on a short line..






2
13.767645864544416
Lenght of line segment before split:  6.883822932272208
while-loop executed for the 1st time
Line length:  13.767645864544416
Line length of 1st subsegment:  6.899999999961537
Lenght of line segment before split:  3.441911466136104

counter:  1
Line ID:  2A
Line length: 6.899999999961537
Line ID:  2B
Line length: 6.8676458645828795
Final line length:  3.4676458643368986
A receiver is placed in a long line..






3
2.565081630078581
A receiver is placed on a short line..

In [16]:
#Convert list of receivers to a dataframe:
r_df = pd.DataFrame(data=[(receiver.id, receiver.building_id, receiver.coord_x, receiver.coord_y, receiver.coord_z)
                          for receiver in receiver_ls], 
                    columns=["receiver_id", "building_id", "coord_x", "coord_y", "coord_z"])
r_df

,receiver_id,building_id,coord_x,coord_y,coord_z
0,0AA,FME_385016b9-dfb2-49d8-8abe-e51f78872af8,117329.870941,6.162040e+06,13.472729
1,0AB,FME_385016b9-dfb2-49d8-8abe-e51f78872af8,117328.509368,6.162037e+06,13.472729
2,0BA,FME_385016b9-dfb2-49d8-8abe-e51f78872af8,117327.147794,6.162035e+06,13.472729
3,0BB,FME_385016b9-dfb2-49d8-8abe-e51f78872af8,117325.786221,6.162032e+06,13.472729
4,1,FME_385016b9-dfb2-49d8-8abe-e51f78872af8,117323.991846,6.162032e+06,13.472729
...,...,...,...,...,...
75,17BA,FME_385016b9-dfb2-49d8-8abe-e51f78872af8,117327.263671,6.162051e+06,13.472729
76,17BB,FME_385016b9-dfb2-49d8-8abe-e51f78872af8,117330.321848,6.162049e+06,13.472729
77,18A,FME_385016b9-dfb2-49d8-8abe-e51f78872af8,117330.942937,6.162047e+06,13.472729
78,18B,FME_385016b9-dfb2-49d8-8abe-e51f78872af8,117329.195814,6.162044e+06,13.472729


In [17]:
#Export dataframe to csv:
#r_df.to_csv(r_out_path+"out.csv") 
r_df.to_csv("receiver_coords.csv") 

<br>
<br>

## Useful links

- [ArcGIS Pro's Analysis tool - Buffer](https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/buffer.htm)
- [ArcGIS Pro's Cartography tool - Simplify Polygon](https://pro.arcgis.com/en/pro-app/3.1/tool-reference/cartography/simplify-polygon.htm)
- [ArcGIS Pro's Data Management tool - Split Line At Vertices](https://pro.arcgis.com/en/pro-app/3.1/tool-reference/data-management/split-line-at-vertices.htm)
- [ArcGIS Pro's Data Management tool - Feature Vertices To Points](https://pro.arcgis.com/en/pro-app/3.1/tool-reference/data-management/feature-vertices-to-points.htm)

<br>
<br>